# BIG DATA ANALYTICS PROGRAMMING : MRJob
### MRJob 라이브러리를 활용한 Map-Reduce 단어수 세기
---

## 1. 라이브러리 설치

In [ ]:
import sys

In [ ]:
#!pip install mrjob
!{sys.executable} -m pip install mrjob

## 2. Word Count 예제
"%%file wordcount.py" 라는 magic 명령어를 활용해 주피터 노트북 내에서 파일 쓰기를 할 수 있습니다

In [ ]:
%%file wordcount.py
from mrjob.job import MRJob
import sys

class MRWordFrequencyCount(MRJob):
    
    def mapper(self, _, line):
        # line = "Hello world"
        yield "chars", len(line)
        yield "words", len(line.split())
        yield "lines", 1

    def reducer(self, key, values):
        # key = "chars"
        # values = [10,10,10]
        yield key, sum(values)


if __name__ == '__main__':
    MRWordFrequencyCount.run()

### map-reduce 로직 이해하기

만약에 다음과 같은 3줄짜리 글이 있을때
```
Hello, this is big data anlytics course.
It is not as difficult as you think.
Good Luck!
```
위의 텍스트를 기준으로 단어 수 세기를 map-reduce를 활용해 계산

예를들어서 mapper가 3개가 생성되었다면,
- mapper1 : Hello, this is big data anlytics course.
    - "chars", 40
    - "words", 7
    - "lines", 1
- mapper2 : It is not as difficult as you think.
    - "chars", 36
    - "words", 8
    - "lines", 1
- mapper3 : Good Luck!
    - "chars", 10
    - "words", 2
    - "lines", 1
    
그리고 reducer가 1개가 생성되었다면,
- "chars", [40,36,10] 을 key-value로 받고
    - "chars", 86을 반환
- "words", [7,8,2] 을 key-value로 받고
    - "words", 17을 반환
- "lines", [1,1,1] 을 key-value로 받고
    - "lines", 3을 반환

### 작은 텍스트 파일로 실행

In [ ]:
!{sys.executable} wordcount.py data_small.txt

### 큰 텍스트 파일로 실행

In [ ]:
!{sys.executable} wordcount.py data.txt

## 3. 고빈출 단어 출력 예제

### 정규식을 활용한, 단어 추출

In [ ]:
import re
WORD_RE = re.compile(r"[\w']+")
for word in WORD_RE.findall("Hello, nice !! [1][2][3]"):
    print(word)

### map-reduce 코드 생성

In [ ]:
%%file wordcount2.py
from mrjob.job import MRJob
from mrjob.step import MRStep
import re

WORD_RE = re.compile(r"[\w']+")


class MRMostUsedWord(MRJob):

    def steps(self):
        return [
            MRStep(mapper=self.mapper_get_words,
                   reducer=self.reducer_count_words),
            MRStep(reducer=self.reducer_find_max_word)
        ]

    def mapper_get_words(self, _, line):
        for word in WORD_RE.findall(line):
            yield (word.lower(), 1)

    def reducer_count_words(self, word, counts):
        yield None, (sum(counts), word)

    def reducer_find_max_word(self, _, word_count_pairs):
        yield max(word_count_pairs)


if __name__ == '__main__':
    MRMostUsedWord.run()

### map-reduce 로직 이해하기

In [ ]:
test = [(10,"dog"),(3, "cat"), (33, "bird")]
max(test)

만약에 다음과 같은 3줄짜리 글이 있을때
```
Hello, this is big data anlytics course.
It is not as difficult as you think.
Good Luck!
```
위의 텍스트를 기준으로 단어 수 세기를 map-reduce를 활용해 계산

예를들어서 mapper가 3개가 생성되었다면,
- mapper1 : Hello, this is big data anlytics course.
    - "hello", 1
    - "this", 1
    - "is", 1
    - ....
- mapper2 : It is not as difficult as you think.
    - "it", 1
    - "is", 1
    - "not", 1
    - ....
- mapper3 : Good Luck!
    - "good", 1
    - "luck", 1
    
그리고 첫번째 reducer 부분에서
- "hello", [1] 을 key-value로 받고
    - None, (1, hello)를 반환
- "is", [1,1] 을 key-value로 받고
    - None, (2, is)를 반환
- ...

그리고 마지막 reducer에서
- 첫번째 reducer의 key가 모두 None이기 때문에, 한쪽으로 다 몰리고
- word_count_pairs = [(1,hello),(2, is)...]
- 마지막으로 word_count_pairs 중 max 값을 return

### 실행

In [ ]:
!{sys.executable} wordcount2.py --runner=inline data_small.txt

In [ ]:
!{sys.executable} wordcount2.py --runner=inline data.txt

## 4. Top 10 단어 출력 예제

In [ ]:
%%file wordcount3.py
from mrjob.job import MRJob
from mrjob.step import MRStep
import re
import sys
WORD_RE = re.compile(r"[\w']+")


class MRTopNWord(MRJob):

    def steps(self):
        return [
            MRStep(mapper=self.mapper_get_words,
                   reducer=self.reducer_count_words),
            MRStep(reducer=self.reducer_find_max_word)
        ]

    def mapper_get_words(self, _, line):
        # yield each word in the line
        for word in WORD_RE.findall(line):
            yield (word.lower(), 1)


    def reducer_count_words(self, word, counts):
        yield None, (sum(counts), word)

    def reducer_find_max_word(self, _, values):
        pass

if __name__ == '__main__':
    MRTopNWord.run()

In [ ]:
!{sys.executable} wordcount3.py --runner=inline data.txt